In [1]:
from shapely.ops import cascaded_union
import matplotlib.pyplot as plt
import geopandas as gpd
import multiprocessing
import pandas as pd
import numpy as np
import skimage.io
import tqdm
import glob
import math
import gdal
import time
import sys
import os
import geopandas as gpd
from shapely import wkt

import matplotlib.pyplot as plt
plt.rcParams.update({'font.size': 16})
import matplotlib as mpl
mpl.rcParams['figure.dpi'] = 300
import matplotlib
# matplotlib.use('Agg') # non-interactive

import solaris as sol
from solaris.utils.core import _check_gdf_load
from solaris.raster.image import create_multiband_geotiff 

# import from data_postproc_funcs
module_path = os.path.abspath(os.path.join('../src/'))
if module_path not in sys.path:
    sys.path.append(module_path)
from sn7_baseline_postproc_funcs import map_wrapper, multithread_polys, \
        calculate_iou, track_footprint_identifiers, \
        sn7_convert_geojsons_to_csv
import solaris.eval.scot as scot

/Midgard/home/hfang/miniconda3/envs/solaris/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/Midgard/home/hfang/miniconda3/envs/solaris/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/Midgard/home/hfang/miniconda3/envs/solaris/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/Midgard/home/hfang

In [2]:
ground_truth = pd.read_csv('/Midgard/home/hfang/temporal_CD/HRNet_SN7_csv/ground_truth.csv')
aois = [z.split('mosaic_')[-1] for z in ground_truth['filename'].values]
ground_truth['aoi'] = aois
timesteps = [z.split('monthly_')[-1].split('_mosaic')[0] for z in ground_truth['filename'].values]
ground_truth['timestep'] = timesteps
ground_truth = ground_truth.drop('filename', 1)

# ground_truth['geometry'] = ground_truth['geometry'].apply(wkt.loads)
# gdf = gpd.GeoDataFrame(ground_truth, geometry = 'geometry')
# print(gdf)

In [3]:
pred = pd.read_csv('/Midgard/home/hfang/temporal_CD/HRNet_SN7/submission_tcr_v1_alpha_09.csv')
aois = [z.split('mosaic_')[-1] for z in pred['filename'].values]
pred['aoi'] = aois
timesteps = [z.split('monthly_')[-1].split('_mosaic')[0] for z in pred['filename'].values]
pred['timestep'] = timesteps
pred = pred.drop('filename', 1)

# pred['geometry'] = pred['geometry'].apply(wkt.loads)
# pdf = gpd.GeoDataFrame(pred, geometry = 'geometry')
# print(pdf)

In [ ]:
# scot.scot_multi_aoi(gdf, pdf, verbose=True)

aois = sorted(list(ground_truth.aoi.drop_duplicates()))
print(aois)
print(len(aois))

# Record the track score, change score and combined score (to csv)
df = pd.DataFrame()

cumulative_score = 0.
all_stats = {}

for i, aoi in enumerate(aois):
    print()
    print('%i / %i: AOI %s' % (i + 1, len(aois), aoi))
    grnd_df_one_aoi = ground_truth.loc[ground_truth.aoi == aoi].copy()
    prop_df_one_aoi = pred.loc[pred.aoi == aoi].copy()
    
    grnd_df_one_aoi['geometry'] = grnd_df_one_aoi['geometry'].apply(wkt.loads)
    gdf = gpd.GeoDataFrame(grnd_df_one_aoi, geometry = 'geometry')
    # print(gdf)
    
    prop_df_one_aoi['geometry'] = prop_df_one_aoi['geometry'].apply(wkt.loads)
    pdf = gpd.GeoDataFrame(prop_df_one_aoi, geometry = 'geometry')
    # print(pdf)

    score_one_aoi, stats_one_aoi = scot.scot_one_aoi(gdf, pdf, threshold=0.25, base_reward=100., beta=2., stats=True, verbose=True)
    cumulative_score += score_one_aoi
    all_stats[aoi] = stats_one_aoi
    
    # Record the score of each AOI
    row1 = {'AOI': aoi, 'Track Score': stats_one_aoi[4], 'Change Score': stats_one_aoi[8], 'Combined Score': score_one_aoi}
    df = df.append(row1, ignore_index=True)

df.to_csv('scot_alpha_09.csv', index=False, header=True)
# Return combined SCOT metric score
score = cumulative_score / len(aois)
print('Overall score: %f' % score)
print(score)
print(all_stats)

['L15-0387E-1276N_1549_3087_13', 'L15-0566E-1185N_2265_3451_13', 'L15-0632E-0892N_2528_4620_13', 'L15-1015E-1062N_4061_3941_13', 'L15-1200E-0847N_4802_4803_13', 'L15-1276E-1107N_5105_3761_13', 'L15-1438E-1134N_5753_3655_13', 'L15-1615E-1206N_6460_3366_13', 'L15-1690E-1211N_6763_3346_13', 'L15-1848E-0793N_7394_5018_13']
10

1 / 10: AOI L15-0387E-1276N_1549_3087_13
Matching footprints
   1: F1 = 0.1955
   2: F1 = 0.2090
   3: F1 = 0.2032
   4: F1 = 0.2032
   5: F1 = 0.2018
   6: F1 = 0.2004
   7: F1 = 0.1998
   8: F1 = 0.2004
   9: F1 = 0.2030
  10: F1 = 0.2060
  11: F1 = 0.2174
  12: F1 = 0.2167
  13: F1 = 0.2068
  14: F1 = 0.2091
  15: F1 = 0.2115
  16: F1 = 0.2156
  17: F1 = 0.2162
  18: F1 = 0.2168
  19: F1 = 0.2185
  20: F1 = 0.2268
  21: F1 = 0.2281
  22: F1 = 0.2168
Identifying mismatches
Tracking:
    Mismatches: 5
      True Pos: 5943
     False Pos: 13498
     False Neg: 31142
   Track Score: 0.2103
Change Detection:
      True Pos: 60
     False Pos: 538
     False Neg: 447
  